In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import numpy as np
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
def load_data():
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Lambda(lambda x: x.view(-1))])

    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    x_train = train_dataset.data.numpy().reshape(-1, 784) / 255.0
    x_test = test_dataset.data.numpy().reshape(-1, 784) / 255.0
    return x_train, x_test

In [ ]:
# Load data
x_train, x_test = load_data()

In [ ]:
# 2. Initialize weights and biases (Students need to implement)
# Example: W1 = np.random.randn(512, 784) * 0.01
# Hint: Use np.zeros for biases

# 3. Define activation functions (Students need to implement)
# def relu(x):
#     return ...

# def sigmoid(x):
#     return ...

# 4. Implement forward pass (Encoder + Decoder) - Students need to implement
# def forward(x):
#     # Encoder: multiple layers with ReLU activation
#     # Decoder: multiple layers with ReLU/Sigmoid activation for reconstruction
#     return z.T, x_reconstructed.T

# 5. Compute Mean Squared Error Loss (Students need to implement)
# def mse_loss(x, x_reconstructed):
#     return ...

# 6. Implement backpropagation and weight updates - Students need to implement
# def backward(x, h1, h2, z, h3, h4, x_reconstructed, lr=0.001):
#     # Compute gradients and update weights and biases
#     pass

# 7. Implement training loop - Students need to implement
# def train(x_train, epochs=100, lr=0.001):
#     # Train the network and collect loss and latent codes
#     return losses, latent_codes, reconstructed_data

# Train the model (Uncomment once students complete the train function)
# losses, latent_codes, reconstructed_data = train(x_train)

# 8. Plot the loss curve
def plot_loss_curve(losses):
    plt.figure(figsize=(8, 6))
    plt.plot(losses, label='MSE Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()
    plt.show()

# Uncomment after training to plot the loss curve
# plot_loss_curve(losses)

# 9. Visualize latent codes using PCA
def visualize_latent_space(latent_codes):
    pca = PCA(n_components=2)
    reduced_codes = pca.fit_transform(latent_codes)
    plt.figure(figsize=(8, 6))
    plt.scatter(reduced_codes[:, 0], reduced_codes[:, 1], s=5, alpha=0.6)
    plt.title('Latent Space Visualization (PCA)')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.show()

# Uncomment after training to visualize latent space
# visualize_latent_space(latent_codes)

# 10. Show original and reconstructed images side by side
def show_images(original, reconstructed, n=10):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()

# Uncomment after training to show images
# show_images(x_train, reconstructed_data)